In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from concurrent.futures import ProcessPoolExecutor
import gc

def get_system_prompt(change_level):
    """Obtener un prompt de sistema detallado según el nivel de cambio"""
    base_prompt = """You are an expert in image-to-image modification with Stable Diffusion XL.
Create a prompt that produces a {change_level} level of change compared to the original image.
Focus on creating a high-quality, detailed prompt for SDXL Refiner 1.0.
Do not include references to the original artist, painting name, or description in the output."""

    level_prompts = {
    "subtle": """
For this SUBTLE modification:
- Maintain the exact composition, subject matter, and artistic intention
- Make minimal adjustments to color temperature, lighting intensity, or texture details
- Ensure the modified image would be recognized as nearly identical to the original
- Focus on enhancing rather than changing elements
- Consider appropriate techniques like increasing detail, adjusting contrast, or enhancing textures

Keywords to consider: refine, enhance, subtle shift, gentle adjustment, nuanced change, detailed, crisp, high-quality""",
    
    "moderate": """
For this MODERATE modification:
- Keep the main composition and subject recognizable
- Transform color schemes, lighting conditions, or artistic techniques
- Add or modify secondary elements while preserving primary subjects
- Create a clear visual difference while maintaining the artwork's essence
- Consider time of day changes, season shifts, or stylistic reinterpretations

Keywords to consider: transform, shift, reinterpret, reimagine, alternative take, artistic variation""",
    
    "radical": """
For this RADICAL modification:
- Completely transform the artistic style, era, or medium
- Dramatically alter color palette, composition, or perspective
- Recontextualize the subject matter in a boldly different setting
- Create a new artistic vision that only conceptually relates to the original
- Consider genre shifts, opposing aesthetics, or unexpected conceptual fusions

Keywords to consider: revolutionize, transpose, transmute, overhaul, profound transformation, reimagined universe"""
    }
    
    return base_prompt.format(change_level=change_level) + level_prompts[change_level] + "\n\nCreate a detailed, vibrant prompt with descriptive adjectives. Maximum 75 words. Avoid words like 'painting' or 'artwork'."

def generate_prompt(model, tokenizer, row, device_id=0):
    """Genera un prompt para Stable Diffusion usando el modelo LLM"""
    # Usar el nivel de cambio desde la columna 'category' de cada fila
    change_level = row['category'].lower()
    system_prompt = get_system_prompt(change_level)
    
    # Información contextual de la pintura
    art_context = f"""Genre: {row['genre']}
Artist: {row['artist']}
Title: {row['painting_name']}
Description: {row['description']}
Change level: {change_level}

Generate a prompt for Stable Diffusion XL Refiner 1.0 to create a variation of this artwork."""

    # Formato para Qwen2.5-7B-Instruct-1M
    full_prompt = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{art_context}<|im_end|>\n<|im_start|>assistant\n"

    with torch.cuda.device(device_id):
        inputs = tokenizer(full_prompt, return_tensors="pt").to(f"cuda:{device_id}")
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=False)  # Cambiado a False para capturar los tokens especiales
    
    # Extraer solo la respuesta del asistente (correctamente)
    assistant_start_tag = "<|im_start|>assistant\n"
    assistant_end_tag = "<|im_end|>"
    
    if assistant_start_tag in response:
        # Obtener el texto después del tag de inicio del asistente
        assistant_part = response.split(assistant_start_tag)[1]
        # Si hay tag de final, cortar ahí
        if assistant_end_tag in assistant_part:
            assistant_response = assistant_part.split(assistant_end_tag)[0].strip()
        else:
            assistant_response = assistant_part.strip()
    else:
        # Fallback: tratar de obtener solo la respuesta nueva
        try:
            input_text_length = len(tokenizer.encode(full_prompt))
            assistant_response = tokenizer.decode(outputs[0][input_text_length:], skip_special_tokens=True).strip()
        except:
            assistant_response = "Error extrayendo la respuesta del modelo."
    
    return assistant_response

def process_batch(model, tokenizer, batch_df, device_id, output_file, processed_so_far=0, save_interval=50):
    """Procesa un batch de datos en la GPU especificada y actualiza el archivo de salida"""
    prompts = []
    
    for i, row in batch_df.iterrows():
        try:
            prompt_text = generate_prompt(model, tokenizer, row, device_id)
            # Verificar si el prompt generado parece contener el formato completo
            if "system\n" in prompt_text or "user\n" in prompt_text:
                print(f"⚠️ Advertencia: Prompt completo detectado en fila {i + processed_so_far}. Limpiando...")
                # Intento de limpieza adicional
                if "<|im_start|>assistant\n" in prompt_text:
                    prompt_text = prompt_text.split("<|im_start|>assistant\n")[1]
                    if "<|im_end|>" in prompt_text:
                        prompt_text = prompt_text.split("<|im_end|>")[0]
                # Eliminamos cualquier texto que contenga los marcadores de sistema o usuario
                lines = [line for line in prompt_text.split("\n") if "system" not in line and "user" not in line]
                prompt_text = "\n".join(lines)
            
            prompts.append(prompt_text)
            
            # Verificar si debemos actualizar el archivo basado en el intervalo
            current_processed = i + 1
            absolute_processed = processed_so_far + current_processed
            
            if current_processed % save_interval == 0 or current_processed == len(batch_df):
                print(f"GPU {device_id} - Processed {absolute_processed} rows total")
                
                # Actualizar el archivo con las filas procesadas hasta ahora
                if os.path.exists(output_file):
                    # Leer el archivo existente y agregar las nuevas filas
                    existing_df = pd.read_csv(output_file)
                    new_rows_df = batch_df.iloc[:current_processed].copy()
                    new_rows_df['sdxl_prompt'] = prompts
                    updated_df = pd.concat([existing_df, new_rows_df])
                    updated_df.to_csv(output_file, index=False)
                    print(f"Archivo actualizado: {output_file} (total: {len(updated_df)} filas)")
                else:
                    # Crear el archivo por primera vez
                    new_rows_df = batch_df.iloc[:current_processed].copy()
                    new_rows_df['sdxl_prompt'] = prompts
                    new_rows_df.to_csv(output_file, index=False)
                    print(f"Archivo creado: {output_file} ({len(new_rows_df)} filas)")
            
            elif i % 10 == 0:
                print(f"GPU {device_id} - Procesado {i+1}/{len(batch_df)} del batch actual (total: {absolute_processed})")
                # Mostrar un ejemplo de prompt limpio para verificación
                
        except Exception as e:
            error_msg = f"ERROR: {str(e)}"
            print(f"Error en fila {i + processed_so_far}: {error_msg}")
            prompts.append(error_msg)
    
    # Agregamos los prompts generados para todas las filas del batch
    batch_df['sdxl_prompt'] = prompts
    
    # Devolver el DataFrame procesado y el número de filas procesadas
    return batch_df, len(batch_df)

def process_csv_chunk(input_file, output_file, device_id, start_idx, end_idx, batch_size=50, save_interval=50):
    """Procesa un segmento del CSV en la GPU especificada y guarda en un solo archivo por GPU"""
    # Carga el modelo en la GPU específica
    model_name = "Qwen/Qwen2.5-7B-Instruct-1M"
    
    with torch.cuda.device(device_id):
        model = AutoModelForCausalLM.from_pretrained(
            model_name, 
            torch_dtype=torch.float16, 
            device_map=f"cuda:{device_id}"
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Lee solo el segmento relevante del CSV
    df = pd.read_csv(input_file)
    chunk = df.iloc[start_idx:end_idx].copy()
    
    # Procesar en batches para manejar grandes volúmenes de datos
    total_batches = (len(chunk) + batch_size - 1) // batch_size
    processed_count = 0
    
    # Si el archivo de salida ya existe (de ejecuciones anteriores), lo eliminamos
    if os.path.exists(output_file):
        os.remove(output_file)
        print(f"Eliminado archivo previo: {output_file}")
    
    for batch_num in range(total_batches):
        batch_start = batch_num * batch_size
        batch_end = min(batch_start + batch_size, len(chunk))
        batch_df = chunk.iloc[batch_start:batch_end]
        
        print(f"GPU {device_id} - Procesando batch {batch_num+1}/{total_batches}")
        processed_batch, batch_processed = process_batch(
            model, tokenizer, batch_df, device_id, output_file, 
            processed_so_far=processed_count, save_interval=save_interval
        )
        
        # Actualizar el archivo de salida con este batch
        if batch_num == 0:
            # Para el primer batch, creamos el archivo
            processed_batch.to_csv(output_file, index=False)
        else:
            # Para batches subsiguientes, actualizamos el archivo existente
            existing_df = pd.read_csv(output_file)
            updated_df = pd.concat([existing_df, processed_batch])
            updated_df.to_csv(output_file, index=False)
        
        processed_count += batch_processed
        print(f"Batch {batch_num+1} completado y agregado a {output_file} (total: {processed_count} filas)")
        
        # Limpiar memoria después de cada batch
        gc.collect()
        torch.cuda.empty_cache()
    
    # Liberar memoria
    del model
    torch.cuda.empty_cache()
    
    return output_file

def main():
    # Configuración
    input_file = "stratify.csv"  # Cambiar al nombre de tu archivo CSV
    output_dir = "resultados_sdxl"
    num_gpus = 2  # Número de GPUs a utilizar
    batch_size = 50  # Tamaño del batch
    save_interval = 50  # Guardar avances cada 50 filas
    max_rows = 200  # Limitar a 200 filas en total para pruebas
    
    # Validación inicial
    print("Iniciando generador de prompts para SDXL Refiner 1.0")
    print(f"Archivo de entrada: {input_file}")
    print(f"Utilizando {num_gpus} GPUs con tamaño de batch: {batch_size}")
    print(f"Guardando progreso cada {save_interval} filas")
    print(f"Procesando un máximo de {max_rows} filas para prueba")
    print("Verificando archivo de entrada...")
    
    # Crear directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Leer el CSV para obtener el número total de filas
    df = pd.read_csv(input_file)
    
    # Limitar a max_rows para pruebas
    if max_rows > 0 and len(df) > max_rows:
        df = df.iloc[:max_rows]
        print(f"Limitando a {max_rows} filas para prueba")
    
    total_rows = len(df)
    print(f"Total de filas a procesar: {total_rows}")
    

    # Dividir el trabajo entre las GPUs
    rows_per_gpu = total_rows // num_gpus
    
    # Definir los archivos de salida
    output_files = [os.path.join(output_dir, f"output_gpu{i}.csv") for i in range(num_gpus)]
    combined_output = os.path.join(output_dir, "combined_output.csv")
    
    # Procesar en paralelo
    with ProcessPoolExecutor(max_workers=num_gpus) as executor:
        futures = []
        for i in range(num_gpus):
            start_idx = i * rows_per_gpu
            end_idx = (i + 1) * rows_per_gpu if i < num_gpus - 1 else total_rows
            
            print(f"GPU {i} procesará filas {start_idx} a {end_idx-1}")
            
            # Enviar el trabajo a cada GPU
            future = executor.submit(
                process_csv_chunk, 
                input_file, 
                output_files[i], 
                i,  # ID de la GPU
                start_idx, 
                end_idx, 
                batch_size,
                save_interval
            )
            futures.append(future)
    
    # Esperar a que todos los procesos terminen
    result_files = [future.result() for future in futures]
    
    # Combinar resultados
    print("Combinando resultados finales...")
    combined_df = pd.concat([pd.read_csv(f) for f in result_files])
    combined_df.to_csv(combined_output, index=False)
    
    print(f"Procesamiento completado. Resultados finales:")
    for i, file in enumerate(output_files):
        file_size = os.path.getsize(file) / (1024 * 1024)  # Tamaño en MB
        print(f"  - GPU {i}: {file} ({file_size:.2f} MB)")
    
    combined_size = os.path.getsize(combined_output) / (1024 * 1024)  # Tamaño en MB
    print(f"  - Combinado: {combined_output} ({combined_size:.2f} MB)")
    print(f"Total de filas procesadas: {len(combined_df)}")

if __name__ == "__main__":
    main()

Iniciando generador de prompts para SDXL Refiner 1.0
Archivo de entrada: stratify.csv
Utilizando 2 GPUs con tamaño de batch: 50
Guardando progreso cada 50 filas
Procesando un máximo de 200 filas para prueba
Verificando archivo de entrada...
Limitando a 200 filas para prueba
Total de filas a procesar: 200
GPU 0 procesará filas 0 a 99
GPU 1 procesará filas 100 a 199
GPU 1 - Procesando batch 1/2
GPU 0 - Procesando batch 1/2
GPU 0 - Procesado 1/50 del batch actual (total: 1)
Ejemplo de prompt generado: Transmute a vibrant still life with a white cockat...
GPU 1 - Procesado 101/50 del batch actual (total: 101)
Ejemplo de prompt generado: Revolutionize the naive primitivism of Fernando Bo...


KeyboardInterrupt: 